# 날씨 빅데이터콘테스트 EDA-weather-data
[날씨 빅데이터 콘테스트]https://bd.kma.go.kr/contest/sub_04.do <br>
[날씨마루] https://bd.kma.go.kr/kma2019/dta/reqst/KBP51001.do?pageNum=5&menuCd=F040201020 <br>
[구글드라이브-데이터]https://drive.google.com/drive/folders/1RO_tBYRFGi-nXWX1Ujb4c8h6gebuCAfp <br>
[기상자료 개방포털] https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36&tabNo=1 <br>

### 분석 주제
공장 내 철강 제품의 결로 발생 예측 모형 개발
* 24, 48시간 후 결로 발생 예측 및 예측경보 시스템 운영방안 제시

### 분석 데이터
> 1. 기상데이터
    * 날씨마루, 기상자료개방 포털
> 2. 현대제철 공장 (당진1공장, 당진2공장)
    * 결로발생여부 : 내/외부 기온, 습도, 코일온도 
        * 1: 발생, 0:미발생
    * 데이터 기간 : 2016.4.1 ~ 2020.3.31

### 정확도 검증
* 2019.4.1 ~ 2020.3.31 중 특정시점 기준 24,48시간 후 결로발생 여부 (0,1)

### 분석 방향
* 이슬점온도가 결로현상과 밀접하게 관련이 있으므로, 24시간/48시간 후의 공장 내부/외부의 이슬점온도를 예측하는 것
    * 이슬점온도 예측을 위해선 온도, 상대습도 예측이 우선
    * 또한 코일온도와 이슬점온도와의 관계에서 결로여부가 결정되므로 코일온도 예측 필요
        

### 테이블 설명
> * plant1_train
    * 공장1의 결로발생 데이터(학습용)
    * 기간 : 2016-04-01 0:00 ~ 2019-03-31 23:50
> * plant2_train
    * 공장2의 결로발생 데이터(학습용)
    * 기간 : 2016-04-01 0:00 ~ 2019-03-31 23:50
> * plant_test
    * 검증용 데이터
    * 2019-04-01 0:00 ~ 2020-03-31 23:50
    * 예측 정확도 검증을 위한 결로발생 여부  비공개

### 날씨데이터 
(참고) 2공장은 당진AWS, 1공장은 신평AWS과 가까움. 서산ASOS는 두공장과 조금 멈<br>

* 당진 = 기온, 습도, 상대습도, 강수, 지면온도
* 신평 = 기온, 습도, 상대습도, 강수, 지면온도
* 서산 = 기온, 습도, 강수, 이슬점온도, 지면온도

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
from itertools import cycle
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [2]:
# jupyter notebook cell 너비 조절
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

In [3]:
# 한글깨짐 문제
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

#### 데이터 불러오기


In [91]:
plant1_train=pd.read_csv('./data/plant1_train.csv',index_col=[0],parse_dates=['plant1_train.mea_ddhr'])
plant2_train=pd.read_csv('./data/plant2_train.csv',index_col=[0],parse_dates=['plant2_train.mea_ddhr'])
plant_test=pd.read_csv('./data/plant_test.csv',index_col=[0],parse_dates=['plant_test.mea_ddhr'])

# 당진
dangjin_humid=pd.read_csv('./data/dangjin_humid.csv',index_col=None)
dangjin_rain=pd.read_csv('./data/dangjin_rain.csv',index_col=None)
dangjin_tem=pd.read_csv('./data/dangjin_tem.csv',index_col=None)

# 신평
shinpyung_humid=pd.read_csv('./data/shinpyung_humid.csv',index_col=None)
shinpyung_rain=pd.read_csv('./data/shinpyung_rain.csv',index_col=None)
shinpyung_tem=pd.read_csv('./data/shinpyung_tem.csv',index_col=None)

#aws=pd.read_csv('./data/aws.csv',index_col=[0])
#forecast=pd.read_csv('./data/forecast.csv',index_col=[0])
# 서산
#seasan=pd.read_csv('./data/seosan.csv',index_col=[0])

# test데이터셋과 동일하게 train 데이터 전처리
plant_train=pd.read_csv('./data/plant_train.csv',index_col=[0])

#### 컬럼명 전처리

In [95]:
# 날씨 컬럼명 정리
dangjin = [dangjin_tem, dangjin_rain, dangjin_humid]

for d in dangjin:
    d.columns = [col.split('.')[1] for col in d.columns]    
    d.rename(columns={'tm':'datetime'}, inplace=True)    

In [96]:
# 공장 컬럼명 정리
plant = [plant1_train, plant2_train, plant_test]

for p in plant:
    p.columns = [col.split('.')[1] for col in p.columns]    
    p.rename(columns={'mea_ddhr':'datetime'}, inplace=True)    

In [28]:
# 다른 방법 
dangjin = [dangjin_tem, dangjin_rain, dangjin_humid]
i = 0
col_n =[]
for d in dangjin:
    col_n.append([])
    for col in d.columns:
        col_n[i].append( col.split('.')[1] )

    print(col_n[i])
    i += 1

['tm', 'aws_id', 'ta', 'ta_hmi', 'ta_avg', 'ta_qcm', 'ta_max', 'ta_max_mi', 'ta_min', 'ta_min_mi']
['tm', 'aws_id', 're_sum', 're_qcm', 'rn_day', 'rn_day_mi', 'rn_hr1', 'rn_hr1_mi', 'rn_60m_max', 'rn_60m_max_mi', 'rn_60m_qcm', 'rn_15m_max', 'rn_15m_max_mi', 'rn_15m_qcm', 'rn01_hr1', 'rn01_hr1_mi']
['tm', 'aws_id', 'hm', 'hm_hmi', 'hm_avg', 'hm_qcm', 'hm_max', 'hm_max_mi', 'hm_min', 'hm_min_mi']


In [97]:
dangjin_tem.head()

,datetime,aws_id,ta,ta_hmi,ta_avg,ta_qcm,ta_max,ta_max_mi,ta_min,ta_min_mi
0,2016-04-05 10:00:00.0,616,13.3,0.0,11.9,60,13.3,-4.0,10.4,NaN
1,2016-04-05 11:00:00.0,616,14.9,0.0,14.1,60,15.3,-12.0,13.3,NaN
2,2016-04-07 20:00:00.0,616,10.9,0.0,11.2,60,11.7,-59.0,10.9,NaN
3,2016-04-13 19:00:00.0,616,15.2,0.0,16.6,60,18.2,-59.0,15.1,NaN
4,2016-04-18 22:00:00.0,616,8.2,0.0,8.7,60,9.2,-59.0,8.2,NaN


In [98]:
dangjin_humid.head()

,datetime,aws_id,hm,hm_hmi,hm_avg,hm_qcm,hm_max,hm_max_mi,hm_min,hm_min_mi
0,2016-04-02 03:00:00.0,616,54.3,0.0,52.2,60,54.6,-2.0,49.1,NaN
1,2016-04-05 12:00:00.0,616,18.2,0.0,19.4,60,24.5,-56.0,15.6,NaN
2,2016-04-05 16:00:00.0,616,23.2,0.0,22.0,60,24.9,-20.0,17.5,NaN
3,2016-04-07 14:00:00.0,616,51.8,0.0,56.3,60,62.2,-59.0,51.8,NaN
4,2016-04-09 04:00:00.0,616,92.3,0.0,92.3,60,92.3,-59.0,92.3,NaN


In [99]:
dangjin_rain.head()

,datetime,aws_id,re_sum,re_qcm,rn_day,rn_day_mi,rn_hr1,rn_hr1_mi,rn_60m_max,rn_60m_max_mi,rn_60m_qcm,rn_15m_max,rn_15m_max_mi,rn_15m_qcm,rn01_hr1,rn01_hr1_mi
0,2016-04-02 03:00:00.0,616,0,60,0.0,0.0,0.0,-60,0.0,-59.0,60,0.0,-59.0,60,NaN,NaN
1,2016-04-04 02:00:00.0,616,0,60,0.0,0.0,0.0,-60,0.0,-59.0,60,0.0,-59.0,60,NaN,NaN
2,2016-04-06 01:00:00.0,616,0,60,0.0,0.0,0.0,-60,0.0,-59.0,60,0.0,-59.0,60,NaN,NaN
3,2016-04-11 11:00:00.0,616,0,60,0.0,0.0,0.0,-60,0.0,-59.0,60,0.0,-59.0,60,NaN,NaN
4,2016-04-13 20:00:00.0,616,0,60,4.0,0.0,0.0,-60,0.0,-59.0,60,0.0,-59.0,60,NaN,NaN


In [100]:
plant1_train.head()

,datetime,tem_in_loc1,hum_in_loc1,tem_coil_loc1,tem_in_loc2,hum_in_loc2,tem_coil_loc2,tem_in_loc3,hum_in_loc3,tem_coil_loc3,tem_out_loc1,hum_out_loc1,cond_loc1,cond_loc2,cond_loc3
1,2016-04-01 00:00:00,16.0,24.0,11.0,14.0,23.0,11.0,13.0,32.0,10.0,9.0,42.0,0.0,0.0,0.0
2,2016-04-01 03:00:00,14.0,28.0,10.0,12.0,32.0,9.0,11.0,42.0,7.0,7.0,59.0,0.0,0.0,0.0
3,2016-04-01 06:00:00,13.0,33.0,10.0,11.0,37.0,9.0,10.0,44.0,7.0,6.0,56.0,0.0,0.0,0.0
4,2016-04-01 09:00:00,13.0,33.0,10.0,11.0,35.0,9.0,10.0,41.0,8.0,18.0,30.0,0.0,0.0,0.0
5,2016-04-01 12:00:00,16.0,28.0,10.0,15.0,27.0,11.0,14.0,30.0,9.0,18.0,20.0,0.0,0.0,0.0


## 이슬점온도와 결로현상
* 코일표면온도가 실내공기 이슬점보다 낮을 때 결로 생김
    * (이슬점 - 표면온도) > 0

In [82]:
def dew_point_c(t_air_c, rel_humidity):
    """
    celcius로 이슬점 계산 함수. 
    
    *attrs
    t_air_c: Series, float. celcius 온도
    rel_humidity: Series, float. 상대습도 %

    *return: float. 이슬점
    """
    A = 17.27
    B = 237.7
    alpha = ((A * t_air_c) / (B + t_air_c)) + np.log(rel_humidity/100.0)
    
    return (B * alpha) / (A - alpha)

In [103]:
plant_train['dew_in'] = dew_point_c(plant_train['tem_in'], plant_train['hum_in'])
plant_train['dew_out'] = dew_point_c(plant_train['tem_out'], plant_train['hum_out'])

In [104]:
temp_cols = ['tem_in', 'hum_in', 'tem_coil', 'tem_out', 'hum_out','dew_in', 'dew_coil', 'dew_out']

In [118]:
plant_train.head()

,datetime,plant,loc,tem_in,hum_in,tem_coil,tem_out,hum_out,cond,x24h_cond,x48h_cond,dew_in,dew_out
0,2016-04-01 00:00:00,1,1,16.0,24.0,11.0,9.0,42.0,0.0,0.0,0.0,-4.562263,-3.224066
1,2016-04-01 00:00:00,1,2,14.0,23.0,11.0,9.0,42.0,0.0,0.0,0.0,-6.806323,-3.224066
2,2016-04-01 00:00:00,1,3,13.0,32.0,10.0,9.0,42.0,0.0,0.0,0.0,-3.310253,-3.224066
3,2016-04-01 03:00:00,1,1,14.0,28.0,10.0,7.0,59.0,0.0,0.0,0.0,-4.223102,-0.461554
4,2016-04-01 03:00:00,1,2,12.0,32.0,9.0,7.0,59.0,0.0,0.0,0.0,-4.184595,-0.461554


In [120]:
cond_y=plant_train.query('cond==1')   # 결로 발생
cond_n=plant_train.query('cond==0') # 결로 미발생

In [121]:
print('결로현상이 발생한 수의 비율 : {} %'.format(round((cond_y.shape[0]/plant_train.shape[0])*100,2)))

결로현상이 발생한 수의 비율 : 0.57 %


In [122]:
print('결로현상 발생하지 않은 수의 비율: {} % '.format(round((cond_n.shape[0]/plant_train.shape[0])*100,2)))

결로현상 발생하지 않은 수의 비율: 98.79 % 


In [131]:
cond_y

,datetime,plant,loc,tem_in,hum_in,tem_coil,tem_out,hum_out,cond,x24h_cond,x48h_cond,dew_in,dew_out
131,2016-04-06 09:00:00,1,3,13.00,84.00,11.00,11.00,45.00,1.0,0.0,1.0,10.358700,-0.476037
178,2016-04-08 09:00:00,1,2,13.00,80.00,10.00,9.00,75.00,1.0,1.0,0.0,9.629510,4.807381
179,2016-04-08 09:00:00,1,3,12.00,82.00,9.00,9.00,75.00,1.0,1.0,0.0,9.021774,4.807381
182,2016-04-08 12:00:00,1,3,13.00,80.00,10.00,12.00,63.00,1.0,0.0,0.0,9.629510,5.174198
197,2016-04-09 03:00:00,1,3,13.00,77.00,10.00,9.00,80.00,1.0,0.0,0.0,9.061264,5.735506
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171669,2019-03-21 02:30:00,2,1,11.68,84.83,10.06,10.43,99.48,1.0,0.0,0.0,9.212235,10.351831
171670,2019-03-21 02:30:00,2,2,12.53,83.91,10.48,10.43,99.48,1.0,0.0,0.0,9.882534,10.351831
171672,2019-03-21 02:40:00,2,1,11.68,84.83,10.18,10.21,99.42,1.0,0.0,0.0,9.212235,10.122943
171675,2019-03-21 02:50:00,2,1,11.59,84.86,10.10,9.94,99.45,1.0,0.0,0.0,9.129254,9.857636


In [146]:
# 이슬점온도가 코일표면온도보다 높을경우 => 100% 결로현상 발생
plant_train[plant_train.dew_in>plant_train.tem_coil]['cond'].value_counts()

1.0    900
Name: cond, dtype: int64

In [144]:
# 이슬점온도가 코일표면온도보다 낮거나 같을경우 => 99.7%가 결로현상 미발생, 단 0.32%만 결로현상 발생.
plant_train[plant_train.dew_in<=plant_train.tem_coil]['cond'].value_counts()

0.0    348361
1.0      1105
Name: cond, dtype: int64